In [202]:
#install.packages("fmsb")
#install.packages("fastDummies")
library("fmsb")
library("fastDummies")

In [149]:
options(scipen=999)

In [306]:
data = read.csv("./Prostate_cancer_micronutrients.csv")

In [307]:
# Creating a boolean column 'cancer in family' 
data$cancerInFamily = as.numeric(data$q4A1 == 2 | data$q4A1 == 3)

# Creating boolean columns for eye colors
data$lightBrownEyes = as.numeric(data$eye4 == "Lt brown")
data$blackOrDarkEyes = as.numeric(data$eye4 == "Black")

# Creating boolean columns for skin colors
data$whiteSkin = as.numeric(data$skin == "V white/white")
data$lightTanSkin = as.numeric(data$skin == "Lt Tan")
data$tanOrDarkOrBlackSkin = as.numeric(data$skin == "Tan" | data$skin == "Dk brown/black")

# Creating boolean columns for sunburn frequency
data$sunburnNever = as.numeric(data$sunburn_fq == "Never")
data$sunburnSeldom = as.numeric(data$sunburn_fq == "Seldom")
data$sunburnOccasionally = as.numeric(data$sunburn_fq == "Occasionally")
data$sunburnFrequently = as.numeric(data$sunburn_fq == "Frequently")

# Creating boolean columns for sun exposure in adulthood
data$sunExposureBelow0.5 = as.numeric(data$adultpw_gp3 == 0)
data$sunExposureOver0.5 = as.numeric(as.numeric(data$adultpw_gp3 == 1) | as.numeric(data$adultpw_gp3 == 2))
data$sunExposure0.5To10 = as.numeric(data$adultpw_gp3 == 1)
data$sunExposure10.1To56 = as.numeric(data$adultpw_gp3 == 2)

In [308]:
columns = c('casectrl', 'age', 'ethnic0', 'school_gp', 'cancerInFamily', 'BMI', 'blackOrDarkEyes', 'lightTanSkin', 'tanOrDarkOrBlackSkin', 'sunburnNever', 'sunburnSeldom', 'sunburnOccasionally', 'sunburnFrequently', 'sunExposureBelow0.5', 'sunExposureOver0.5', 'sunExposure0.5To10', 'sunExposure10.1To56')
for (c in columns) {
    print(c)
    print(toString(sum(is.na(data[c]))))
}

[1] "casectrl"
[1] "11"
[1] "age"
[1] "11"
[1] "ethnic0"
[1] "11"
[1] "school_gp"
[1] "14"
[1] "cancerInFamily"
[1] "14"
[1] "BMI"
[1] "123"
[1] "blackOrDarkEyes"
[1] "19"
[1] "lightTanSkin"
[1] "17"
[1] "tanOrDarkOrBlackSkin"
[1] "17"
[1] "sunburnNever"
[1] "32"
[1] "sunburnSeldom"
[1] "32"
[1] "sunburnOccasionally"
[1] "32"
[1] "sunburnFrequently"
[1] "32"
[1] "sunExposureBelow0.5"
[1] "11"
[1] "sunExposureOver0.5"
[1] "11"
[1] "sunExposure0.5To10"
[1] "11"
[1] "sunExposure10.1To56"
[1] "11"


It seems that we _really_ need to impute BMI

In [309]:
data$BMI[is.na(data$BMI)] = mean(data$BMI, na.rm = TRUE)

In [310]:
print("BMI")
print(toString(sum(is.na(data['BMI']))))

[1] "BMI"
[1] "0"


## Modeling

### Model 1

In [311]:
model1 = glm(casectrl ~ age + ethnic0 + school_gp + cancerInFamily + BMI + blackOrDarkEyes + lightTanSkin + tanOrDarkOrBlackSkin, data=data)
summary(model1)


Call:
glm(formula = casectrl ~ age + ethnic0 + school_gp + cancerInFamily + 
    BMI + blackOrDarkEyes + lightTanSkin + tanOrDarkOrBlackSkin, 
    data = data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.95889  -0.34729  -0.01471   0.35723   1.10961  

Coefficients:
                      Estimate Std. Error t value       Pr(>|t|)    
(Intercept)          -0.938693   0.245712  -3.820       0.000151 ***
age                   0.015820   0.002492   6.348 0.000000000500 ***
ethnic0              -0.076629   0.031774  -2.412       0.016246 *  
school_gp             0.148865   0.023110   6.441 0.000000000284 ***
cancerInFamily        0.190574   0.042429   4.492 0.000008837052 ***
BMI                  -0.014035   0.005277  -2.660       0.008080 ** 
blackOrDarkEyes       0.264318   0.045473   5.813 0.000000011132 ***
lightTanSkin          0.232851   0.050644   4.598 0.000005447690 ***
tanOrDarkOrBlackSkin  0.267210   0.058587   4.561 0.000006453399 ***
---
Signif

In [312]:
# Printing OR and 95% CI:
exp(cbind(OR = coef(model1), confint(model1)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.3911386,0.2416464,0.6331127
age,1.0159455,1.0109954,1.0209199
ethnic0,0.9262332,0.8703112,0.9857486
school_gp,1.1605167,1.1091233,1.2142915
cancerInFamily,1.2099437,1.1133960,1.3148635
BMI,0.9860628,0.9759166,0.9963146
blackOrDarkEyes,1.3025424,1.1914754,1.4239627
lightTanSkin,1.2621939,1.1429262,1.3939075
tanOrDarkOrBlackSkin,1.3063147,1.1646037,1.4652695


### Model 2

In [313]:
model2 = glm(casectrl ~ age + ethnic0 + school_gp + cancerInFamily + BMI + skin2 + sunburnSeldom + sunburnOccasionally + sunburnFrequently + sunExposureOver0.5 + sunExposure0.5To10 + sunExposure10.1To56, data=data)
summary(model2)


Call:
glm(formula = casectrl ~ age + ethnic0 + school_gp + cancerInFamily + 
    BMI + skin2 + sunburnSeldom + sunburnOccasionally + sunburnFrequently + 
    sunExposureOver0.5 + sunExposure0.5To10 + sunExposure10.1To56, 
    data = data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.95972  -0.36166  -0.05893   0.38840   0.97784  

Coefficients: (1 not defined because of singularities)
                     Estimate Std. Error t value       Pr(>|t|)    
(Intercept)         -0.948567   0.255680  -3.710       0.000232 ***
age                  0.017033   0.002606   6.536 0.000000000165 ***
ethnic0             -0.082647   0.030506  -2.709       0.006992 ** 
school_gp            0.134607   0.025180   5.346 0.000000140853 ***
cancerInFamily       0.194012   0.044460   4.364 0.000015737444 ***
BMI                 -0.015584   0.005490  -2.839       0.004726 ** 
skin2                0.115345   0.019378   5.952 0.000000005181 ***
sunburnSeldom        0.010284   0.04

In [303]:
# Printing OR and 95% CI:
exp(cbind(OR = coef(model2), confint(model2)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.3872955,0.2346430,0.6392596
age,1.0171791,1.0119969,1.0223879
ethnic0,0.9206765,0.8672420,0.9774034
school_gp,1.1440870,1.0889947,1.2019665
cancerInFamily,1.2141113,1.1127921,1.3246555
BMI,0.9845368,0.9740001,0.9951875
skin2,1.1222608,1.0804366,1.1657041
sunburnSeldom,1.0103368,0.9160902,1.1142793
sunburnOccasionally,1.1339126,0.9976811,1.2887463
sunburnFrequently,1.1895580,1.0305154,1.3731461


## Manual way to calculate OR and 95% CI:

### Eye colour

In [135]:
a = cbind(c(209,25), 
          c(169,97))
a

209,169
25,97


In [136]:
result = oddsratio(a, conf.level=0.95)
or = result['estimate']
ci = result['conf.int']

           Disease Nondisease Total
Exposed        209        169   378
Nonexposed      25         97   122
Total          234        266   500


In [137]:
print(or)
print(ci)

$estimate
[1] 4.798343

$conf.int
[1] 2.956929 7.786490
attr(,"conf.level")
[1] 0.95

